Try different learning rates for GD vs MRGD

* Use cosine similarity as metric


In [ ]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiresolver_pie import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average, F2fluxconverter
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 64
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9
max_shift = 32
lmbda = 1e-30
noise = 1e100
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
#image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
LR_list = [0,0,0,2e-2,3e-2,5e-2,1e-1] # Currently Liepschitz is used - this is not being used

In [ ]:
type_alg = "gd"  # "gd" or "mrgd"
if type_alg == "gd":
    I_out = 10*np.array([0, 0, 0, 0, 0,0,80])
elif type_alg == "mrgd":
    I_out =10*np.array([0, 0, 0, 8, 5,5,62])

model = MultiResSolver(multires, 
                        loss, 
                        LR = LR_list,
                        I_in = I_in,
                        I_out = I_out,
                        tol = tol,
                        tol_in = tol_in,
                        cycle = cycle,
                        l1_type = "l1_row",
                        gt = image_tensor_,
                        scaler = 16,
                        early_stopping=9,
                        tol_vals= [5, 0 ,1500],)
model.solve_multigrid()
model.print_time()


In [ ]:
# Part for visualizing the loss

loss  = extract_data(model.measures["loss"])
np.save("loss_pie.npy", loss)
plt.figure(figsize=(10, 6),dpi =300)
plt.semilogy(np.cumsum(model.iter_times),loss[:len(model.iter_times)])
plt.xlabel("Time (s)")
plt.ylabel("Loss")
plt.title("Loss vs Time")
plt.grid()
plt.show()

In [ ]:
# Part for visualizing the reconstructed image

mean_img = np.mean(image)
phase = torch.angle(model.c_k[0,0,:,:].to('cpu'))
phase = phase.numpy()
phase = unwrap_2d(phase)
plt.imshow(phase, cmap='gray')